# Efficient Net tested on 3 different datasets

## Import data

In [7]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Transformations to apply to the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # normalize images
])

### Cifar 10

In [8]:
# Download CIFAR-10 dataset
trainset_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                download=True, transform=transform)
testset_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=False,
                                               download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# Create data loaders
train_loader_cifar10 = DataLoader(trainset_cifar10, batch_size=32, shuffle=True)
val_loader_cifar10 = DataLoader(trainset_cifar10, batch_size=32, shuffle=False)

### Stanford Cars

In [6]:
# Download Stanford Cars dataset
trainset_stf_cars = torchvision.datasets.StanfordCars(root='./data', split='train',
                                                download=True, transform=transform)
testset_stf_cars = torchvision.datasets.StanfordCars(root='./data', split='test',
                                               download=True, transform=transform)

HTTPError: HTTP Error 404: Not Found

In [ ]:
# Create data loaders
train_loader_stf_cars = DataLoader(trainset_stf_cars, batch_size=32, shuffle=True)
val_loader_stf_cars = DataLoader(testset_stf_cars, batch_size=32, shuffle=False)

### EuroSAT

In [ ]:
# Download CIFAR-10 dataset
trainset_euroSAT = torchvision.datasets.euroSAT(root='./data', train=True,
                                                download=True, transform=transform)
testset_euroSAT = torchvision.datasets.euroSAT(root='./data', train=False,
                                               download=True, transform=transform)

In [ ]:
# Create data loaders
train_loader_euroSAT = DataLoader(trainset_euroSAT, batch_size=32, shuffle=True)
val_loader_euroSAT = DataLoader(testset_euroSAT, batch_size=32, shuffle=False)

## Model

### Declare lightning model

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from timm import create_model
import lightning as L

class EfficientNetPL(L.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.model = create_model('efficientnet_b0', pretrained=True)
        num_ftrs = self.model.classifier.in_features
        self.model.classifier = nn.Linear(num_ftrs, num_classes)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        self.log('val_loss', loss)
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == labels).sum().item()
        self.log('val_accuracy', correct / len(labels))

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

c:\Users\benoi\AppData\Local\pypoetry\Cache\virtualenvs\projet-8inf974-qm87_-2b-py3.9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Cifar 10

In [11]:
# Instantiate the model
model = EfficientNetPL(num_classes=10)

# Train using PyTorch Lightning Trainer
trainer = L.Trainer(max_epochs=10)
trainer.fit(model, train_loader_cifar10, val_loader_cifar10)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\benoi\AppData\Local\pypoetry\Cache\virtualenvs\projet-8inf974-qm87_-2b-py3.9\lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | EfficientNet     | 4.0 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
4.0 M     Trainable params
0         Non-t

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\benoi\AppData\Local\pypoetry\Cache\virtualenvs\projet-8inf974-qm87_-2b-py3.9\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


c:\Users\benoi\AppData\Local\pypoetry\Cache\virtualenvs\projet-8inf974-qm87_-2b-py3.9\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 0:   0%|          | 4/1563 [00:01<12:13,  2.13it/s, v_num=3]

c:\Users\benoi\AppData\Local\pypoetry\Cache\virtualenvs\projet-8inf974-qm87_-2b-py3.9\lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
